In [1]:
# First attempt at logistic regression, step by step.

import pandas as pd
import numpy as np
import os
#import seaborn as sns
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split

In [30]:
df = pd.read_csv('../../../../GitHub_Data/Dota2Matches/match_outcomes.csv')

In [31]:
# Messy. Let's just work with player info. drop parser_version and start_time, for starters
df.drop(['parser_version', 'start_time'], axis=1, inplace=True)
df = df.melt(id_vars=['match_id','win','rad'], \
    value_vars=['account_id_0', 'account_id_1', 'account_id_2', 'account_id_3', 'account_id_4'], \
        var_name='PlayerNumber', value_name='PlayerAccountID')
df.head()

,match_id,win,rad,PlayerNumber,PlayerAccountID
0,1636204962,1,0,account_id_0,34549
1,1636204962,0,1,account_id_0,0
2,1636322679,0,0,account_id_0,0
3,1636322679,1,1,account_id_0,-97530201
4,1637385965,1,0,account_id_0,0


In [32]:
dfPlayerRatings = pd.read_csv('../../../../GitHub_Data/Dota2Matches/player_ratings.csv')
df = pd.merge(df,dfPlayerRatings,how='left', left_on='PlayerAccountID', right_on='account_id')
df.head()

,match_id,win,rad,PlayerNumber,PlayerAccountID,account_id,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,1636204962,1,0,account_id_0,34549,34549,10,17,25.496820,5.277160
1,1636204962,0,1,account_id_0,0,0,1608398,3315071,25.000000,8.333333
2,1636322679,0,0,account_id_0,0,0,1608398,3315071,25.000000,8.333333
3,1636322679,1,1,account_id_0,-97530201,-97530201,1,1,26.880745,8.118755
4,1637385965,1,0,account_id_0,0,0,1608398,3315071,25.000000,8.333333


In [33]:
# so the blank slot doesn't really mean anything. All those 0s are just empty spots where somebody dropped.
# We'll get rid of them.

df.drop(['account_id'], axis=1, inplace=True)
df = df[df['PlayerAccountID'] != 0]
df.head()

,match_id,win,rad,PlayerNumber,PlayerAccountID,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,1636204962,1,0,account_id_0,34549,10,17,25.496820,5.277160
3,1636322679,1,1,account_id_0,-97530201,1,1,26.880745,8.118755
6,1637623870,1,0,account_id_0,-123447796,22,47,24.867150,3.543024
7,1637623870,0,1,account_id_0,-108454938,28,46,32.028790,3.659563
8,1637739731,0,0,account_id_0,320093,9,15,26.680175,5.869468


In [34]:
# The question becomes - how to create a 'team' from the individuals?
# For now, let's just add the trueskill_mu for each player and ignore the sigma
# why add and not average? mostly because this is just a test and i'm curious of the results

df.drop(['PlayerNumber', 'PlayerAccountID', 'total_wins', 'total_matches'], axis=1, inplace=True)
df.head()

,match_id,win,rad,trueskill_mu,trueskill_sigma
0,1636204962,1,0,25.496820,5.277160
3,1636322679,1,1,26.880745,8.118755
6,1637623870,1,0,24.867150,3.543024
7,1637623870,0,1,32.028790,3.659563
8,1637739731,0,0,26.680175,5.869468


In [35]:
dfRad = df[df['rad'] == 1].copy()
dfDire = df[df['rad'] == 0].copy()

dfRad.rename(columns={'win':'Rad_Win'}, inplace = True)
dfRad.rename(columns={'trueskill_mu': 'radiant_trueskill_mu_sum', 'trueskill_sigma': 'radiant_trueskill_sigma_mean'},inplace=True)
dfRad['radiant_trueskill_mu_mean'] = dfRad['radiant_trueskill_mu_sum']
dfRad.drop(['rad'], axis = 1, inplace=True)

dfDire.rename(columns={'trueskill_mu': 'dire_trueskill_mu_sum', 'trueskill_sigma': 'dire_trueskill_sigma_mean'},inplace=True)
dfDire.drop(['win','rad'], axis = 1, inplace=True)
dfDire['dire_trueskill_mu_mean'] = dfDire['dire_trueskill_mu_sum']

print(dfRad.head())
print(dfDire.head())

      match_id  Rad_Win  radiant_trueskill_mu_sum  \
3   1636322679        1                 26.880745   
7   1637623870        0                 32.028790   
11  1637843670        0                 29.610686   
14  1638064585        0                 22.376919   
20  1638310272        0                 21.488357   

    radiant_trueskill_sigma_mean  radiant_trueskill_mu_mean  
3                       8.118755                  26.880745  
7                       3.659563                  32.028790  
11                      5.286551                  29.610686  
14                      7.685111                  22.376919  
20                      7.561265                  21.488357  
      match_id  dire_trueskill_mu_sum  dire_trueskill_sigma_mean  \
0   1636204962              25.496820                   5.277160   
6   1637623870              24.867150                   3.543024   
8   1637739731              26.680175                   5.869468   
10  1637843670              24.473918

In [37]:
dfPreparedData = pd.merge(dfRad,dfDire,how='inner', on='match_id')
dfPreparedData = pd.pivot_table(dfPreparedData, index=['match_id'], \
    aggfunc={'Rad_Win': np.max, 'radiant_trueskill_mu_sum': np.sum, 'radiant_trueskill_mu_mean': np.mean, 'radiant_trueskill_sigma_mean': np.mean, \
        'dire_trueskill_mu_sum': np.sum, 'dire_trueskill_mu_mean': np.mean, 'dire_trueskill_sigma_mean': np.mean})
dfPreparedData.head(10)

,Rad_Win,dire_trueskill_mu_mean,dire_trueskill_mu_sum,dire_trueskill_sigma_mean,radiant_trueskill_mu_mean,radiant_trueskill_mu_sum,radiant_trueskill_sigma_mean
match_id,,,,,,,
1636204962,0,25.357769,228.219917,5.307879,24.991777,224.925992,4.645305
1636322679,1,25.429459,152.576754,5.577277,29.280546,175.683275,7.656243
1637385965,0,28.605807,57.211613,4.346619,23.977755,47.955511,7.380554
1637623870,0,27.253514,545.070286,5.717296,24.122732,482.454634,6.343619
1637739731,1,25.028565,400.457043,6.378492,28.637235,458.195754,6.417469
1637843670,0,24.467287,489.345735,5.577694,24.137326,482.746520,6.577699
1637995977,1,23.875327,286.503921,7.030672,24.409829,292.917945,5.196412
1638064585,0,25.850216,413.603454,6.388290,21.416274,342.660388,6.631475
1638252636,0,25.240946,201.927567,4.836085,23.948786,191.590289,4.494508


In [38]:
dfPreparedData.shape

(827280, 7)

In [39]:
dfPreparedData.to_csv('../../../../GitHub_Data/Dota2Matches/preparedMatchSkill.csv')